# Freshman On-Track (FOT) Intervention Recommender
### A Standalone Proof-of-Concept

This notebook demonstrates a working PoC for an AI-powered intervention recommender.

**This notebook is designed to run in Google Colab.** It contains all the code needed to set up its environment, download the project from GitHub, and run the demonstration.

## 1. Universal Setup

This cell is the "magic" that prepares the entire environment. It intelligently detects where it's running and performs the correct setup automatically.

Here's what happens when you run the next cell:
1.  **Define Project Source**: We specify the official GitHub repository for this project so it's clear where the code comes from.
2.  **Detect Environment**: The notebook checks if it's running inside the local project folder or as a standalone file.
3.  **Prepare Environment**: A helper script is called to do the heavy lifting:
    - If **local**, it uses your existing project files.
    - If **standalone**, it clones the repository and installs all dependencies for you.

After running this one cell, the environment will be ready for the demonstration.

In [1]:
import sys
from pathlib import Path

# --- Define Project Source ---
REPO_URL = "https://github.com/chuckfinca/fot-intervention-recommender.git"
PROJECT_DIR_NAME = "fot-intervention-recommender"

# print("🚀 Setting up the environment...")

# # --- Clone the Repository & Install Dependencies ---
# !git clone -q {REPO_URL}
# %pip install -q -r {PROJECT_DIR_NAME}/requirements.txt

# # --- Configure Python Path ---
# project_path = Path.cwd() / PROJECT_DIR_NAME
# src_path = project_path / "src"
# sys.path.insert(0, str(src_path))

# print("\n🎉 Environment is ready!")


print("🚀 Setting up LOCAL development environment...")

# We assume the notebook is in 'notebooks/'. The project root is one level up.
project_path = Path.cwd().parent

# Configure Python Path to use the local 'src' directory
src_path = project_path / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"  - Using local project root: {project_path}")
print("\n🎉 Local environment is ready!")

🚀 Setting up LOCAL development environment...
  - Using local project root: /Users/charlesfeinn/Developer/job_applications/fot-intervention-recommender

🎉 Local environment is ready!


## 2. Load the Knowledge Base

With the environment bootstrapped, we can now import our project's modules and load the data. The `project_path` variable ensures we find the file correctly.

In [2]:
# Import the functions from our custom Python package (now in the path)
from fot_recommender.rag_pipeline import (
    load_knowledge_base,
    initialize_embedding_model,
    create_embeddings,
    create_vector_db,
    search_interventions,
)

# Build the path to the knowledge base using the universal project_path variable
kb_path = project_path / "data" / "processed" / "knowledge_base_final_chunks.json"

# Load the knowledge base
knowledge_base_chunks = load_knowledge_base(str(kb_path))

print(f"Successfully loaded {len(knowledge_base_chunks)} intervention chunks.")
knowledge_base_chunks[0]

/Users/charlesfeinn/Developer/job_applications/fot-intervention-recommender/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully loaded 27 intervention chunks.


{'title': 'Strategy: Leadership Roles',
 'source_document': 'NCS_OTToolkit_2ndEd_October_2017_updated.pdf',
 'fot_pages': 'Pages: 44',
 'content_for_embedding': 'Title: Strategy: Leadership Roles. Content: Principal Role:\n• Implementation: Reviews and interrogates interim freshman success-related data in light of Success Team goals, and strategizes with team leadership around next steps',
 'original_content': 'Principal Role:\n• Implementation: Reviews and interrogates interim freshman success-related data in light of Success Team goals, and strategizes with team leadership around next steps'}

In [4]:
# --- Build the RAG Pipeline Components ---
#
# Now, we will initialize the core components of our RAG pipeline.
# 1. Embedding Model: We'll load the model that converts text into vectors.
# 2. Vector Embeddings: We'll encode all our knowledge base chunks.
# 3. Vector Database: We'll create a FAISS index for fast searching.
#
# These components will be stored in variables for the rest of the notebook to use.

# 1. Initialize the embedding model
embedding_model = initialize_embedding_model()

# 2. Create vector embeddings for the knowledge base
embeddings = create_embeddings(knowledge_base_chunks, embedding_model)

# 3. Set up the FAISS vector database
vector_db = create_vector_db(embeddings)

Initializing embedding model: all-MiniLM-L6-v2...
Model initialized successfully.
Creating embeddings for 27 chunks...


Batches:   0%|                                                                                                                                                                                                        | 0/1 [00:00<?, ?it/s]/Users/charlesfeinn/Developer/job_applications/fot-intervention-recommender/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.03s/it]

Embeddings created successfully.
Creating FAISS index with dimension 384...
FAISS index created with 27 vectors.


## 5. Try It Yourself: Enter Your Own Query

Now it's your turn. The system is ready to accept a new query.

Describe the challenges of a hypothetical student in the text box below. For example, you could try:
- "A student is missing a lot of school and their grades are suffering."
- "This freshman has good attendance but is failing math and science and seems disengaged."
- "A student has multiple behavior incidents and is struggling to connect with teachers."

The RAG system will perform a new semantic search and return the top 3 interventions from the knowledge base that best match your description.

In [5]:
from fot_recommender.utils import display_recommendations

# Prompt the user to enter their own query
user_query = input("Enter a description of a student's challenges: ")

if user_query:
    print("\n🔍 Searching for interventions based on your query...")

    # Perform a new search using the user's input
    custom_recommendations = search_interventions(
        query=user_query,
        model=embedding_model,
        index=vector_db,
        knowledge_base=knowledge_base_chunks,
        k=3,
    )

    # Display the new results using our helper function
    display_recommendations(custom_recommendations)

else:
    print("\nNo query entered. Skipping custom search.")

Enter a description of a student's challenges:  asdf



🔍 Searching for interventions based on your query...

Searching for top 3 interventions for query: 'asdf...'
Found 0 relevant interventions.

No relevant interventions were found for this query.


In [ ]:
import shutil
from pathlib import Path

# The path to the project directory we created at the start
project_path_to_clean = Path.cwd() / "fot-recommender-poc-workspace"

if project_path_to_clean.exists():
    print(f"The project directory '{project_path_to_clean}' was found.")

    # Ask for user confirmation before deleting anything
    response = input(
        "Would you like to delete the git repository folder that was downloaded during the running of this notebook? (y/n): "
    )

    if response.lower().strip() == "y":
        try:
            shutil.rmtree(project_path_to_clean)
            print(f"✅ Successfully deleted '{project_path_to_clean}'.")
        except OSError as e:
            print(f"Error: {e.strerror}. Could not delete the directory.")
    else:
        print("Cleanup skipped.")
else:
    print("Project directory not found. Nothing to clean up.")